# Adobe Lightroom-like implementation for image enhancement

In this project, the user will be able to use several parameters to enhance his image in a simulated **Adobe Lightroom** environment.

Every image will be transformed in the RGB space, this is used to treat similarly BNW images or already existing RGB images.
The different possible changes are the following: luminance, saturation and color regulation, noise reduction, equalisaton and autoleveling. 

In [30]:
%matplotlib widget
from ipywidgets import *
from skimage.io import imread,imsave,imshow
import numpy as np
from matplotlib import pyplot as plt
from skimage.color import rgb2hsv, hsv2rgb
from skimage.filters.rank import median
from skimage.morphology import disk

#Take for argument: an image 
#Transform the image to RGB
def to_rgb(im):
    if (np.array(im).ndim == 2):
        im_rgb = np.stack((im, im, im), axis=2).astype('uint8')
    else:
        im_rgb = im
    return im_rgb

#Take for argument: an image
#Generate its cumulative histogram
def cumul_hist(im):
    cumul_hist = np.zeros((256,))
    c = 0
    for v in range(256):
        c += (im==v).sum()
        cumul_hist[v] += c
    cumul_hist /= cumul_hist.max()
    return cumul_hist

#Take for arguments: an image, a channel c and a factor f
#Compute a LUT for a specific channel c which is later use to increase or decrease
#all the values of the image on this channel by a factor f
#This is an irreversible change, we lose information when we set values to 0 or 255
def color_adjust_channel_rgb(im, c, f):
    LUT = np.arange(256)   
    LUT += f
    for i in range(256):
        if LUT[i] < 0:
            LUT[i] = 0
        if LUT[i] > 255:
            LUT[i] = 255
    return LUT[im[:, :, c]]

#Take for argument: an image
#Apply a LUT to equalise the image
def equalise_rgb(im):
    for i in range(3):
        LUT = (255*cumul_hist(im[:,:,i])).astype('uint8')
        im[:,:,i] = LUT[im[:,:,i]]
    return im

#Take for arguments: an image and the wanted level
#Apply an auto-leveling LUT, the low and high cut percentage are symetrical and are define by level
def auto_level(im, level):
    h = cumul_hist(im)
    low_perc = level/100
    high_perc = 1-low_perc
    for v in range(256):
        if h[v] > low_perc: break
    Tmin = v-1
    for v in range(256):
        if h[255-v] < high_perc: break
    Tmax = (255-v)+1
    LUT = np.arange(256)
    LUT[:Tmin] = 0
    LUT[Tmax:] = 255
    LUT[Tmin:Tmax] = (255/(Tmax-Tmin))*(LUT[Tmin:Tmax]-Tmin)
    for i in range(3):
        im[:,:,i] = LUT[im[:,:,i]]
    return im 

#Take for arguments: an image and a factor f
#Increase or decrease value of a RGB image by a factor f on every channel
#This affects the global luminance of the image
#This is irreversible as it uses the color_adjust_channel_rgb(...) function
def adjust_luminance_rgb(im, f):
    for i in range(3):
        im[:,:,i] = color_adjust_channel_rgb(im, i, f)
    return im

#Takes for arguments: an image and a size for the disk
#Apply a median filter on every channel
#Used to reduce the noise
def median_rgb(im, disk_size):
    for i in range(3):
        im[:,:,i] = median(im[:,:,i], disk(disk_size))
    return im

#Takes for arguments: a HSV image, two integers i and j, a channel (axis) and a factor f
#Increase or decrease a value of a specific channel of the HSV image by a factor
#This is an irreversible change, we lose information when we set values to 0 or 1.
def apply(im_hsv, i, j, axis, f):
    if f == 0:
        return im_hsv
    elif im_hsv[i, j, axis] + f < 0:
        im_hsv[i, j, axis] = 0
    elif im_hsv[i, j, axis] + f > 1.:
        im_hsv[i, j, axis] = 1.
    else:
        im_hsv[i, j, axis] += f
    return im_hsv

#Take for arguments: an image and various factors
#Use to change the [Hue, Saturation, Value] of the reds, greens and blues
#As H ranges from 0 to 1 after a rgb2hsv(im) and in its definition every colors correspond to a value between
#0 and 360, 1 degree = 1/360
#Per definition: reds (of RGB) range from 0 to 60 and from 300 to 360, normalised it corresponds to [0;1/6]u[5/6;1]
#greens (of RGB) range from 60 to 180, normalised it corresponds to [1/6;1/2]
#blues (of RGB) range from 180 to 300, normalised it corresponds to [1/2;5/6]
#The same approach has been used for S and V
#s_f (saturation) and l_f (luminance) are global factors that are applied for every colors
#This is irreversible as it uses the apply(...) function
def hsv_color_based_adjust(im, h_r_f, h_g_f, h_b_f, s_r_f, s_g_f, s_b_f, v_r_f, v_g_f, v_b_f, s_f, l_f):
    im_hsv = rgb2hsv(im)
    for i in range(im_hsv.shape[0]):
        for j in range(im_hsv.shape[1]):
            if 0.<=im_hsv[i, j, 0]<=1/6 or 5/6<=im_hsv[i, j, 0]<=1.:
                im_hsv = apply(im_hsv, i, j, 0, h_r_f)
                im_hsv = apply(im_hsv, i, j, 1, s_r_f)
                im_hsv = apply(im_hsv, i, j, 2, v_r_f)
            elif 1/6<=im_hsv[i, j, 0]<=1/2:
                im_hsv = apply(im_hsv, i, j, 0, h_g_f)
                im_hsv = apply(im_hsv, i, j, 1, s_g_f)
                im_hsv = apply(im_hsv, i, j, 2, v_g_f)
            elif 1/2<=im_hsv[i, j, 0]<=5/6:
                im_hsv = apply(im_hsv, i, j, 0, h_b_f)
                im_hsv = apply(im_hsv, i, j, 1, s_b_f)
                im_hsv = apply(im_hsv, i, j, 2, v_b_f)
            im_hsv = apply(im_hsv, i, j, 1, s_f)
            im_hsv = apply(im_hsv, i, j, 2, l_f)
    return (hsv2rgb(im_hsv)*255).astype('uint8')    

#Take for arguments: a RGB image and all the adaptable parameters
#Apply all the parameters to the image
#As multiple functions defined here above are irreversible, this function create a copy of the image
#to apply all the parameters so the original image is not corrupted
def lightroom_rgb(im, lumi_rgb, lumi_hsv, auto, level, equal, saturation, R_color, G_color, B_color, Hue_R, 
                  Hue_G, Hue_B, Saturation_R, Saturation_G, Saturation_B, Value_R, Value_G, Value_B, 
                  median_size):
    im_out = im.copy()
    if lumi_rgb != 0:
        im_out = adjust_luminance_rgb(im_out, lumi_rgb)
    if auto and level != 0:
        im_out = auto_level(im_out, level)
    if equal:
        im_out = equalise_rgb(im_out)
    if R_color != 0:
        im_out[:,:,0] = color_adjust_channel_rgb(im_out, 0, R_color)
    if G_color != 0:
        im_out[:,:,1] = color_adjust_channel_rgb(im_out, 1, G_color)
    if B_color != 0:
        im_out[:,:,2] = color_adjust_channel_rgb(im_out, 2, B_color)
    if Hue_R!=0 or Hue_G!=0 or Hue_B!=0 or Saturation_R!=0 or Saturation_G!=0 or Saturation_B!=0 or Value_R!=0 or Value_G!=0 or Value_B!=0 or lumi_hsv!=0 or saturation!=0:
        im_out = hsv_color_based_adjust(im=im_out, h_r_f=Hue_R, h_g_f=Hue_G, h_b_f=Hue_B, s_r_f=Saturation_R, 
                                        s_g_f=Saturation_G, s_b_f=Saturation_B, v_r_f=Value_R, v_g_f=Value_G, 
                                        v_b_f=Value_B, s_f=saturation, l_f=lumi_hsv)
    if median_size != 0:
        im_out = median_rgb(im_out, median_size)
    return im_out

#This is a class which has one attribute and one method
#The attribute is an array of the modified image
#The method is used to set the array with the latest modifications in order to save it
class Im_modif:
    def __init__(self, array):
        self.array = array
    def setArray(self, im):
        self.array = im

Concerning the coding part, use of LUT was the primary goal but for the HSV manipulations as the H, S and V values are ranging from 0 to 1 and as I did not know the exact number of levels between these two boundaries a choice based on the definition of HSV was made.

<img src="./HSVexplain.png" width="250px" />

This image was taken from Wikipedia. The H values range by definition from $0^{\circ}$ to $360^{\circ}$. Therefore instead of using a LUT, I have implemented a double loop which goes through every pixel in the image to set the wanted values by factors proportional to $1/360$ (the determined distance between to color in the HSV space). The same step distance has been used for the S and V channel. 

This has the drawback to be slower than the optimised LUT method.

To use this code, first enter an image. You should know that although this code will process BNW images, it has  a better and more interesting use for color images.

In [31]:
im = imread('etretat.jpg')

In [15]:
im = imread('astronaut_noisy.jpg')

In [16]:
im = imread('airplane.jpg')

The second step is the widget set up and the update(...) function definition. As some changes can be irreversible, the lightroom_rgb(...) function makes a copy of the original image and apply every changes to the copied version so it does not corrupt the original image.

**Remark** concerning the overall implementation, any HSV transformations take time (roughly 15 to 20 seconds) to be applied on and this is true for each update while at least one of those HSV parameters are different than 0. As previously said LUT use could be an improvement of the already existing method or the use of the previous transormations so it does not to be computed anymore.

In [34]:
#Initialise the array in which will be stored the modifications to the image
im_modif = Im_modif(np.zeros(im.shape))

#Set up plot
fig, ax = plt.subplots(figsize=(6, 4))
ax.imshow(im)
ax.axis('off')

#Set up the widgets
lumi_rgb=widgets.IntSlider(value=0, min=-125, max=125, step=5, description='Lumi RGB', continuous_update=False)
lumi_hsv=widgets.FloatSlider(value=0, min=-1., max=1., step=5/360, description='Lumi HSV', continuous_update=False) 
auto=widgets.Checkbox(value=False, description='Auto')
level=widgets.IntSlider(value=0, min=0, max=100, step=1, description='Level (%)', continuous_update=False)
equal=widgets.Checkbox(value=False, description='Equal')
saturation=widgets.FloatSlider(value=0, min=-1., max=1., step=5/360, description='Saturation', continuous_update=False) 
R_color=widgets.IntSlider(value=0, min=-255, max=255, step=5, description='R color', continuous_update=False)
G_color=widgets.IntSlider(value=0, min=-255, max=255, step=5, description='G color', continuous_update=False) 
B_color=widgets.IntSlider(value=0, min=-255, max=255, step=5, description='B color', continuous_update=False) 
Hue_R=widgets.FloatSlider(value=0, min=-1., max=1., step=5/360, description='Hue R', continuous_update=False) 
Hue_G=widgets.FloatSlider(value=0, min=-1., max=1., step=5/360, description='Hue G', continuous_update=False) 
Hue_B=widgets.FloatSlider(value=0, min=-1., max=1., step=5/360, description='Hue B', continuous_update=False) 
Saturation_R=widgets.FloatSlider(value=0, min=-1., max=1., step=5/360, description='Saturation R', continuous_update=False)
Saturation_G=widgets.FloatSlider(value=0, min=-1., max=1., step=5/360, description='Saturation G', continuous_update=False) 
Saturation_B=widgets.FloatSlider(value=0, min=-1., max=1., step=5/360, description='Saturation B', continuous_update=False) 
Value_R=widgets.FloatSlider(value=0, min=-1., max=1., step=5/360, description='Value R', continuous_update=False)
Value_G=widgets.FloatSlider(value=0, min=-1., max=1., step=5/360, description='Value G', continuous_update=False) 
Value_B=widgets.FloatSlider(value=0, min=-1., max=1., step=5/360, description='Value B', continuous_update=False) 
median_size=widgets.IntSlider(value=0, min=0, max=20, step=1, description='Median', continuous_update=False)

#Set up the layout of the widgets
def make_boxes():
    vbox1 = widgets.VBox([widgets.Label('General'), auto, level, equal, median_size, saturation])
    vbox2 = widgets.VBox([widgets.Label('RGB'), lumi_rgb, R_color, G_color, B_color])
    vbox3 = widgets.VBox([widgets.Label('HSV'), lumi_hsv, Hue_R, Hue_G, Hue_B, Saturation_R, Saturation_G, Saturation_B, 
                          Value_R, Value_G, Value_B])
    return vbox1, vbox2, vbox3
 
vbox1, vbox2, vbox3 = make_boxes()
ui = widgets.HBox([vbox1, vbox2, vbox3])

#Take all the adaptable parameters as arguments and update the plotted image accordingly
def update(lumi_rgb, lumi_hsv, auto, level, equal, saturation, R_color, G_color, B_color, Hue_R, Hue_G, Hue_B, Saturation_R, Saturation_G, 
           Saturation_B, Value_R, Value_G, Value_B, median_size):
    ax.imshow(lightroom_rgb(to_rgb(im), lumi_rgb, lumi_hsv, auto, level, equal, saturation, R_color, G_color, B_color, Hue_R, Hue_G, Hue_B, 
                            Saturation_R, Saturation_G, Saturation_B, Value_R, Value_G, Value_B, median_size))
    im_modif.setArray(lightroom_rgb(to_rgb(im), lumi_rgb, lumi_hsv, auto, level, equal, saturation, R_color, G_color, B_color, Hue_R, Hue_G, Hue_B, 
                                    Saturation_R, Saturation_G, Saturation_B, Value_R, Value_G, Value_B, median_size))

out = widgets.interactive_output(update, {'lumi_rgb':lumi_rgb, 'lumi_hsv':lumi_hsv, 'auto':auto, 'level':level,'equal':equal, 'saturation':saturation, 
                                         'R_color':R_color, 'G_color':G_color, 'B_color':B_color, 'Hue_R':Hue_R, 'Hue_G':Hue_G, 
                                         'Hue_B':Hue_B, 'Saturation_R':Saturation_R, 'Saturation_G':Saturation_G, 
                                         'Saturation_B':Saturation_B, 'Value_R':Value_R, 'Value_G':Value_G, 'Value_B':Value_B, 'median_size':median_size})
display(ui, out)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Output()

The changes made to the image can be saved and displayed at anytime during the process of enhancement. This is done using the "im_modif" object previously (in the update(...) function) updated.

In [19]:
im_to_save = im_modif.array
imsave('name.jpg', im_to_save)

Finally, every function can be used independently. Here is an example of the lightroom_rgb(...) function with fixed parameters for the "etretat.jpg" image:

In [35]:
im = imread('etretat.jpg')
im_modif = lightroom_rgb(to_rgb(im), -10, 0, False, 0, True, 5/360, 0, 0, -5, 10/360, 0, -10/360, 10/360, 10/360, 0, -5/360, 0, 0, 0)

plt.figure()
plt.imshow(im_modif)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …